# Modeling A Particle In the Gravity Field of an Oblate Planet
### Cee Gould

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
%matplotlib osx    
from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict

cmaps = OrderedDict()

$V(r,u) = \frac{GM}{r} [1 - \sum (\frac{a}{r})^{2n} J_{2n} P_{2n}(\mu)] $ , only using n = 1
<br \>
<br \>
*$P_{n2} = \frac{1}{2}(3x^{2} - 1)$
<br \>
*$J_{2n} = 2$
<br \>
$\mu = \frac{z}{r}$
<br \>
$V(r,u) = \frac{GM}{r} [1 - (\frac{a}{r})^{2} *  J_{2} * (\frac{1}{2} (3x^{2} - 1)*
(\mu)] $ , only using n = 1
<br \>

In [ ]:
#----------// User Inputs // ----------
x = 2
y = -1.5
z = 0.5

vx = 0
vy = 12
vz = 0

b = 0.5 #short radius
a = 2    #long radius

ms = 3e+6  #planet mass
mp = 1  #particle mass

In [ ]:
# //---------- Oblate Planet Function ----------
def V(t, X):
    
    x = X[0]
    y = X[1]
    z = X[2]
    
    r = X[0:3]
    v = X[3:6]
    
    # // PERIODIC ORBITS ABOUT AN OBLATE SPHEROID, MACMILLAN 1910
    # https://www.ams.org/journals/tran/1910-011-01/S0002-9947-1910-1500856-2/S0002-9947-1910-1500856-2.pdf
    
    dVx = - (k**2) * ms * mp / np.linalg.norm(r)**3 * (1 + (3/10)*b**2*(x**2 + y**2 - 4*z**2)/ np.linalg.norm(r)**4  * mu**2 ) * x
    dVy = - (k**2) * ms * mp / np.linalg.norm(r)**3 * (1 + (3/10)*b**2*(x**2 + y**2 - 4*z**2)/ np.linalg.norm(r)**4  * mu**2 ) * y
    dVz = - (k**2) * ms * mp / np.linalg.norm(r)**3 * (1 + (3/10)*b**2*(3*(x**2 + y**2) - 2*z**2)/ np.linalg.norm(r)**4  * mu**2 ) * z
    
    #// First attempt
#     dVx = -((G*M)/(np.linalg.norm(r)**2))*x + 3*((6*M*a**2)/(np.linalg.norm(r)**4))*x
#     dVy = -((G*M)/(np.linalg.norm(r)**2))*y + 3*((6*M*a**2)/(np.linalg.norm(r)**4))*y
#     dVz = -((G*M)/(np.linalg.norm(r)**2))*z - ((30*G*M*a**2*z**2)/np.linalg.norm(r)**6) + 3*((6*M*a**2)/(np.linalg.norm(r)**4))*z
    
    #F = -G * ms * mp /np.linalg.norm(r)**3 * r #Newtons Method
    
    
    F = np.array([dVx, dVy, dVz])
    acceleration    = F / mp
        
    return np.concatenate((v, acceleration))

In [ ]:
# // ----------Constants ----------//
year  = 3.154e+7 #s
au = 1.496e+8 #km

k = 0.01720209895 #rad/day
#k = np.sqrt(G*Msun)

G  = k**2 #1e-6
M = ms + mp

#G  = 6.674e-20  #km^3/kg s^2

grav_param = G * M

a_earth = 6371 #km  # equatorial radius
b_earth = 6356.8 #km #short length of earth #polar radius 
m_earth = 5.972e24 #kg
m_moon = 1 #7.35e22 #kg

In [ ]:
r = np.array([x,y,z])
v = np.array([vx,vy,vz])

print('r = ', r)
print('v = ', v)

mu = 1 - b/a      #oblateness of the spheroid 0 < mu < 1. mu = 0 is a sphere. 
print("mu = ", mu)

In [ ]:
#/// ----------Calculate orbital elements ----------/// 
#Fundamentals of Astrodynamics and Applications, by Vallado, 2007

h = np.cross(r,v)         # angular momentum
K = np.array([0, 0, 1])
nhat=np.cross(K,h)       #node vector

#eccentricity vector
evec = ((np.linalg.norm(v)**2 - grav_param /np.linalg.norm(r))*r - np.dot(r,v)*v)/ grav_param 
# evec = (np.linalg.norm(abs(v))**2/mu - 1/np.linalg.norm(abs(r)))*r - (np.dot(r,v)*v / mu) 

#evec = np.array([e_p9, 0, 0])
e = np.linalg.norm(evec)

if e == 0:
    orbit = 'circular orbit'
if e > 0 and e < 1:
    orbit = 'elliptical orbit'
if e == 1:
    orbit ='parabolic orbit'
if e > 1:
    orbit = 'hyperbolic orbit'
print('e = ', e, orbit)

energy = np.linalg.norm(v)**2/2 - grav_param /np.linalg.norm(r) #Specific mechanical energy
if energy < 0: #absolute value of potential energy is larger than kinetic
    bound = 'bound'
if energy > 0: #kinetic energy is larger than the absolute value of the potential energy
    bound = 'unbound'
print('E = ', energy, bound)

if abs(e) != 1: 
    a = - grav_param/(2*energy)
    #p = a*(1-e**2)
    p = np.sqrt(abs(a)**3)
else:
    #p = np.linalg.norm(h)**2/mu
    a = 'inf'
    p = 'inf'
    
print('a = ', a)         #semi major axis


print('p = ', p)        #period

i = np.arccos(h[2]/np.linalg.norm(h))  #inclination
print('i = ', i)        

Omega = np.arccos(nhat[0]/np.linalg.norm(nhat))  #swivel: the angle from the principal direction to the ascending node, 0° ≤ Ω < 360°
if nhat[1] < 0:
   Omega = 360-Omega
print('Omega = ', Omega)

argp = np.arccos(np.dot(nhat,evec)/(np.linalg.norm(nhat)*e)) # argument of perigee, ω: 0° ≤ ω < 360
if evec[2]<0:
   argp = 360-argp
print('argp = ', argp)

#nu_0 = nu_p9
nu_0 = np.arccos(np.dot(evec,r)/(e*np.linalg.norm(r)))  # True anomaly, ν, is the angle along the orbital path from perigee to the spacecraft’s position vector r.  0 <= v < 360°
if np.dot(r,v)<0:
   nu = 360 - nu_0
print('initial nu = ', nu_0, 'degrees')  # changes with time, location of the spacecraft in its orbit

In [ ]:
#// ----------Set Time ----------//
dt = .001
tmax = 4.0
tt = []
t = 0

In [ ]:
# //---------- RUN THE CODE!! ----------// 
xt = []
yt = []
zt = []
vx = []
vy = []
vz = []

X = np.concatenate((r,v))

%matplotlib osx    

while(t < tmax):

    r = X[0:3]
    v = X[3:6]
    
    
    xt.append(X[0])
    yt.append(X[1])
    zt.append(X[2])
    
    
    #X = X + V(X) * dt    #EULERS METHOD
    
    f1 = V(t       ,X          )    #4th order RUNGE KUTTA METHOD
    f2 = V(t+dt/2.0,X+f1*dt/2.0)
    f3 = V(t+dt/2.0,X+f2*dt/2.0)
    f4 = V(t+dt    ,X+f3*dt    )

    X = X + (f1 + 2.0*f2 + 2.0*f3 + f4) / 6.0 * dt
    
    
    tt.append(t)
    t += dt
    
    #// -----------Real Time Plot --------------//
    
#     %matplotlib osx 
#     fig = plt.figure(figsize=(5,5))
    
#     plt.subplot(2, 1, 1)
#     plt.plot(yt, xt, 'r-')
#     plt.xlabel('Y axis')
#     plt.ylabel('X axis')
#     plt.plot(0,0,'*',mfc='w',ms=10)
#     plt.gca().set_aspect('equal', adjustable='box')
    
    
    
#     fig.add_subplot(2, 1, 2)
#     plt.plot(yt, zt, 'r-')
#     plt.xlabel('Y axis')
#     plt.ylabel('Z axis')
#     plt.plot(0,0,'*',mfc='w',ms=10)
#     plt.gca().set_aspect('equal', adjustable='box')
    
#     plt.draw()
#     plt.show()
#     plt.pause(0.0005) 


In [ ]:
#----------// time plots //----------

# plt.subplot(3, 1, 1)

# plt.plot(tt,xt, 'r-')
# plt.title('Height of X over time')

# fig.add_subplot(3, 1, 2)
# plt.plot(tt,yt, 'r-')
# plt.title('Height of Y over time')

# fig.add_subplot(3, 1, 3)
# plt.plot(tt,zt, 'r-')
# plt.title('Height of Z over time')


# plt.show()
# plt.pause(5)  
# plt.close()

In [ ]:
#---------- // 3D Plot // ----------

# // Create 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect("equal")
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

# ax.set_xlim(-1,1)
# ax.set_ylim(-1, 1)
# ax.set_zlim(-1, 1)

#// Orbit //
XT = np.array(xt)
YT = np.array(yt)
ZT = np.array(zt)
orbit = ax.plot(XT, YT, ZT, color = 'b') 
  
#// Sphere //

u, v = np.mgrid[0:2*np.pi:200j, 0:np.pi:100j]   
x = 0.1 * a * np.cos(u)*np.sin(v)
y = 0.1* a * np.sin(u)*np.sin(v)
z = 0.1 * b * np.cos(v)
planet = ax.plot_surface(x, y, z, color="b", cmap='gist_earth')


# // Rotating Plot //

# for angle in range(0, 360):
#     orbit = ax.plot(XT, YT, ZT, color = 'b')  
#     planet = ax.plot_surface(x, y, z, color="b", cmap='gist_earth')
    
#     ax.autoscale_view(True,True,True)     
#     ax.view_init(20, angle)
#     plt.pause(.00001)
    